# Matrix multiply speedup: 63,000 X?
Let's code it in pure python, then compare with Numpy (C) on CPU, Pytorch and Tensorflow on GPU.

Let's write down two matrices and multiply them using Python.

This will also serve as a five minute introduction to Python!

The matrices we will use are the 2x2 matrices below.
$$ 
A =\begin{bmatrix}
1 & 2 \\
3 & 4 \\
\end{bmatrix}
\text{ }
B =\begin{bmatrix}
5 & 6 \\
7 & 8 \\
\end{bmatrix}
$$
A list is a Python data type that is commonly used when working with arrays. In the frameworks we will use, one common way to define a matrix is as a list of lists.

In [1]:
A = [[1,2],[3,4]] 

In [2]:
A

[[1, 2], [3, 4]]

In [3]:
type(A)

list

In [4]:
for row in A:
  print(row)

[1, 2]
[3, 4]


In [5]:
B = [[5,6],[7,8]]

In [6]:
B[0]

[5, 6]

In [7]:
type(B[0])

list

In [8]:
B[1]

[7, 8]

In [9]:
row1_B = B[0]
row2_B = B[1]
print(row1_B)
print(row2_B)

[5, 6]
[7, 8]


In [10]:
element_1_2 = B[0][1]
element_1_2

6

In [11]:
for i in range(2):
    print(i)

0
1


In [12]:
def matrix_multiply(A,B):
    AB = [[0,0],[0,0]] # Create matrix of zeros
    for i in range(2):
        for j in range(2):
            AB[i][j] = sum(A[i][k]*B[k][j] for k in range(2))
    return AB

In [13]:
help(matrix_multiply)

Help on function matrix_multiply in module __main__:

matrix_multiply(A, B)



In [14]:
matrix_multiply(A,B)

[[19, 22], [43, 50]]

The result is 
$$ 
\begin{bmatrix}
1 & 2 \\
3 & 4 \\
\end{bmatrix}
\cdot
\begin{bmatrix}
5 & 6 \\
7 & 8 \\
\end{bmatrix}
= \begin{bmatrix}
19 & 22 \\
43 & 50 \\
\end{bmatrix}
$$

Below we write a Python function to multiply square matrices of any size. We are not interested in optimizing its runtime, rather we focus on writing a function that reads like the mathematical operation. Specifically the fact that the $i,j$ entry in the result $AB$ is given by the formula below.
$$
AB_{i,j} = \sum_{k=1}^n A_{i,k}B_{k,j}
$$

In [15]:
def matrix_multiply(A,B):
    """Multiply square matrices and return result"""
    m = len(A) 
    AB = [[0]*m for i in range(m)] # create AB as matrix of zeros then fill in results 
    
    for i in range(m):
        for j in range(m):
            AB[i][j] = sum(A[i][k]*B[k][j] for k in range(m))
    
    return AB
        

Let's create a matrix initialized with random values.

In [54]:
import random


In [17]:
random.randrange(10) #random number between 0 and m

8

In [18]:
m = 850 # matrix size 850
A = [[random.randrange(m) for j in range(m)] for i in range(m)]
B = [[random.randrange(m) for j in range(m)] for i in range(m)]

In [19]:
%%timeit
matrix_multiply(A,B)

1min 4s ± 547 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Implement in Numpy
Numpy is a Python package with numerical arrays similar to those found in R, C and Fortran. So they are like the lists above but all elements must be of the same numeric type such as a float or int. Numpy sends matrix calculations to numerical libraries written in C (or if you want, to languages linkable to C, such as Fortran).   

In [20]:
import numpy as np
A_np = np.array(A)
B_np = np.array(B)

In [21]:
A_np

array([[619, 706, 792, ..., 163, 389, 801],
       [653, 208, 833, ..., 495, 381, 278],
       [ 38,  77, 609, ..., 769, 593,  90],
       ...,
       [431, 531,  39, ..., 204, 780, 262],
       [704, 669, 542, ...,  86, 164, 154],
       [771, 229,  77, ...,   4, 539, 164]])

In [22]:
%%timeit
np.matmul(A_np,B_np)

693 ms ± 9.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


So how much faster is that?
Let's say it was 600ms. The Python calculation was about 60 seconds, or 60000 ms. Hence Numpy was 100 X faster! 

# PyTorch
First let's try it on the CPU. It should be about the same speed as Numpy as it should call to the same C libraries.
Then we will move the data to the GPU and perform the matrix multiply there.


In [23]:
import torch

In [24]:
A_t = torch.tensor(A)
B_t = torch.tensor(B)

In [25]:
%%timeit
torch.matmul(A_t,B_t)

81.7 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Surprisingly this was a 10X improvement over Numpy. As far as I know there is no reason to expect such an improvement in general. Performance for matrix multiply strongly depends on the [BLAS](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms) library to which the NumPy and Pytorch implementation relies on. It may be Numpy is using an older library.
In any case, we are at a 1000 X improvement over pure Python!

# Let's try on the GPU
First we check that GPU's are available to us.
One way to do this is from the command line using the command `nvidia-smi`.

In [26]:
!nvidia-smi

Wed Nov  3 12:48:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:1B:00.0 Off |                  Off |
| 30%   31C    P8    27W / 300W |      0MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

We can also use utilities in Pytorch.

In [27]:
torch.cuda.is_available()

True

How many (I requested 1).

In [28]:
torch.cuda.device_count()

1

What GPU?

In [29]:
torch.cuda.get_device_name()

'NVIDIA RTX A6000'

How much GPU memory currently used?

In [30]:
print('Memory allocated:', round(torch.cuda.memory_allocated()/1024**3,4), 'GB')
print('Memory reserved:   ', round(torch.cuda.memory_reserved()/1024**3,4), 'GB')

Memory allocated: 0.0 GB
Memory reserved:    0.0 GB


First we move the matrices from the CPU memory to the GPU memory.

In [31]:
A_t.to('cuda')
B_t.to('cuda')

tensor([[ 89, 701, 616,  ..., 521, 488, 227],
        [753, 216, 821,  ..., 569, 595, 363],
        [741,  88, 416,  ..., 500, 712, 758],
        ...,
        [118, 553, 492,  ..., 490, 582, 138],
        [ 86,  79, 353,  ..., 200, 308, 547],
        [476, 444,  81,  ..., 108, 428, 243]], device='cuda:0')

In [32]:
print('Memory reserved:   ', round(torch.cuda.memory_reserved()/1024**3,4), 'GB')

Memory reserved:    0.0215 GB


In [33]:
%%timeit
# ensure that context initialization finish before you start measuring time
torch.matmul(A_t,B_t)
torch.cuda.synchronize()

79.5 ms ± 922 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


No speed up? Let's investigate the data type. 

In [34]:
A_t.dtype

torch.int64

Another consideration is the matrix size. Let's investigate matrix multiply for bigger matrices. We create two numpy arrays below of size `n` by `n`.

In [35]:
n = 10000
C = np.random.rand(n,n)
D = np.random.rand(n,n)
C_t = torch.tensor(C).to("cuda")
D_t = torch.tensor(D).to("cuda")

How much bigger are these matrices?

In [36]:
print('Memory reserved:   ', round(torch.cuda.memory_reserved()/1024**3,4), 'GB')

Memory reserved:    1.5137 GB


What is the speed without GPU?

In [37]:
%%timeit
np.matmul(C,D)

20.6 s ± 169 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


And on GPU?

In [38]:
%%timeit
torch.matmul(C_t,D_t)
torch.cuda.synchronize()

3.56 s ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We add `torch.cuda.synchronize()` to tell Python to wait for the GPU computation to finish before completing the timing. Otherwise, as soon as the computation is handed off to the GPU, `timeit` reports it finished, giving incorrect timings. 

Another way to time the GPU computation is with `torch.cuda.Event()`. Let's see how the timings compare.

In [39]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
torch.matmul(C_t,D_t)
end.record()

# Waits for everything to finish running
torch.cuda.synchronize()

print(start.elapsed_time(end)/1000)

3.575721923828125


We get the same result, so that is comforting!

Next, before we do new computations with new variables we will remove these matrices from the GPU. 

In [40]:
del C_t
del D_t
torch.cuda.empty_cache()

Let's check memory now.

In [41]:
print('Memory reserved:   ', round(torch.cuda.memory_reserved()/1024**3,4), 'GB')

Memory reserved:    0.0195 GB


## Optimizations: Single and half precision, matrix sizing
Next we explore changing the datatype type and the matrix size. As we have 336 Tensor cores each optimized for 4x4 block multiplication, let's try a matrix of size 1344x1344, where 1344=4*336. We will loop the computation to have it perform 1000 matrix multiplies and time that.

In [42]:
n = 1344
C = np.random.rand(n,n)
D = np.random.rand(n,n)
C_t = torch.tensor(C)
D_t = torch.tensor(D)

In [43]:
print(C.dtype)
print(C_t.dtype)

float64
torch.float64


Next we create double precision on the GPU.

In [44]:
C_td = torch.cuda.DoubleTensor(C)
D_td = torch.cuda.DoubleTensor(D)

In [45]:
print(C_td.dtype)

torch.float64


Next, single precision on the GPU.

In [46]:
C_ts = torch.cuda.FloatTensor(C)
D_ts = torch.cuda.FloatTensor(D)

In [47]:
print(C_ts.dtype)

torch.float32


Finally, half precision on the GPU.

In [48]:
C_th = torch.cuda.HalfTensor(C)
D_th = torch.cuda.HalfTensor(D)

In [49]:
print(C_th.dtype)

torch.float16


Let's try the computation now with these different precisions.

First double precision on the CPU. Recall, a similar computation earlier was already ~1000 X faster than pure Python.

In [50]:
%%timeit
for i in range(100):
    torch.matmul(C_t,D_t)
torch.cuda.synchronize()

5.24 s ± 93.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Next, double precision on the GPU.

In [51]:
%%timeit
for i in range(100):
    torch.matmul(C_td,D_td)
torch.cuda.synchronize()

899 ms ± 47 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


So here we see a 100 X improvement over the CPU, hence we are at ~100,000 X pure Python.

Now, single precision on the GPU.

In [52]:
%%timeit
for i in range(100):
    torch.matmul(C_ts,D_ts)
torch.cuda.synchronize()

11.3 ms ± 34.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now we are at ~100X double precision on GPU.  

Finally, half precision on the GPU.

In [53]:
%%timeit
for i in range(100):
    torch.matmul(C_th,D_th)
torch.cuda.synchronize()

4.81 ms ± 2.72 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We are at 2X single precision. For a grand total of 20,000,000 X pure Python!